In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plot
import numpy as np
from numpy import genfromtxt
import csv
import sys, os, re
import pathlib

print(tf.__version__)

2.0.0


In [2]:
# The CANlab/WAVi Pain Study is Pre-Formatted to resemble BIDS neuroimaging formatting
# If your study does not abide to the following structure, please revisit previous scripts
# .../StudyRepo
# -------------> /raw
# -------------------> /*.art
# -------------------> /*.eeg
# -------------------> /*.evt
# -------------> /contigs
# -------------------> /train          2:1 train:test
# -------------------> /test

In [3]:
os.getcwd()

'/home/claytonjschneider/science/WAViMedEEG'

In [4]:
try:
    os.chdir("CANlabStudy/contigs")
    directory=os.getcwd()
except:
    print("I couldn't find the contigs folder.\n")
    directory = input("Please give the full path of the contigs folder: ")
    os.chdir(directory)

In [5]:
train_path = pathlib.Path('train/')
test_path = pathlib.Path('test/')

In [6]:
# load in image paths for training set

import random
train_image_paths = list(train_path.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)

train_count = len(train_image_paths)
print("You have", train_count, "training images.")

# load in image paths for testing set?

test_image_paths = list(test_path.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
random.shuffle(test_image_paths)

test_count = len(test_image_paths)
print("You have", test_count, "testing images.")

You have 1683 training images.
You have 843 testing images.


In [7]:
#list the available labels
label_names = sorted(item.name for item in train_path.glob('*/') if item.is_dir())
print("Labels discovered:", label_names)

Labels discovered: ['chronic', 'p300']


In [8]:
#assign an index to each label
label_to_index = dict((name, index) for index, name in enumerate(label_names))
print("Label indices:", label_to_index)

Label indices: {'chronic': 0, 'p300': 1}


In [9]:
#create a list of every file and its index label
train_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                     for path in train_image_paths]

test_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in test_image_paths]

In [10]:
def load_numpy_stack(paths):
    numpy_stack = []
    for path in paths:
        numpy_stack.append(genfromtxt(path, delimiter=","))
    return(numpy_stack)

In [11]:
train_arrays = load_numpy_stack(train_image_paths)
test_arrays = load_numpy_stack(test_image_paths)

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_arrays, train_image_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_arrays, test_image_labels))

In [13]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [14]:
def kerasModel(learn, beta1, beta2):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(1250, 19)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learn, beta_1=beta1, beta_2=beta2),
                 loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])
    return(model)

In [19]:
def fitModel(model):
    model.fit(train_dataset, epochs=10)
    return(model)

In [20]:
def testModel(model):
    metrics = model.evaluate(test_dataset)
    return(metrics)

In [21]:
rates = np.arange(0.01, 0.1, 0.01)
beta1s = np.arange(0.9, 0.99, 0.01)
beta2s = np.arange(0.99, 0.999, 0.001)

# learning rate does well around 0.01 or 0.02
# beta1 does well around 0.91
# beta2 does well around 0.997

rate = float(input("What's my learning rate? "))
beta1 = float(input("What's my beta1? "))
beta2 = float(input("What's my beta2? "))

myresults=[]

compiled = kerasModel(rate, beta1, beta2)
fitted = fitModel(compiled)
results = testModel(fitted)

What's my learning rate? 0.09
What's my beta1? 0.9
What's my beta2? 0.99
Epoch 1/10
27/27 [==============================] - 2s 57ms/step - loss: 3571.6277 - sparse_categorical_accuracy: 0.5668
Epoch 2/10
27/27 [==============================] - 1s 43ms/step - loss: 2619.2733 - sparse_categorical_accuracy: 0.7296
Epoch 3/10
27/27 [==============================] - 1s 42ms/step - loss: 4610.3451 - sparse_categorical_accuracy: 0.7748
Epoch 4/10
27/27 [==============================] - 1s 45ms/step - loss: 6931.3407 - sparse_categorical_accuracy: 0.8283
Epoch 5/10
27/27 [==============================] - 1s 42ms/step - loss: 3575.2889 - sparse_categorical_accuracy: 0.8948
Epoch 6/10
27/27 [==============================] - 1s 43ms/step - loss: 2346.7024 - sparse_categorical_accuracy: 0.9245
Epoch 7/10
27/27 [==============================] - 1s 42ms/step - loss: 2026.9624 - sparse_categorical_accuracy: 0.9352
Epoch 8/10
27/27 [==============================] - 1s 46ms/step - loss: 3941.19

In [22]:
print("Loss: ", results[0], "\nAccuracy: ", results[1])

Loss:  34708.94391741072 
Accuracy:  0.7698695


In [34]:
beta1s

array([0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98])